In [ ]:
%pip install -r requirements.txt

In [2]:
from control2048 import Control
from game import GameOf2048
import numpy as np
from random import choices

In [3]:
class DTAgent:
    # tries to do the move with the best expected survival (not losing),
    # in contrast of going for the best expected score.
    # since the state space is so huge,
    # we look ahead to some max depth.
    # the depth changes depending on how much tiles are numbered.
    # less tiles -> less risk of losing -> less lookahead to survive
    # and even better!
    # more tiles -> less future states to look at -> more depth analyzed in the same time
    # since we are going to look for everything to some depth, we don't care about search order


    def __init__(self, maxDepth=10):
        self.maxDepth = maxDepth
        self.cache = {}
        self.moves = 0
    
    def nextMove(self, board):
        # we assume the board can be moved
        moves = []
        for move in range(4):
            self.game = GameOf2048()
            # moves are ints,
            # this is explained in the GameOf2048 class definition



            moved, _ = self.game.transform(board, move)
            if not np.array_equal(board, moved):
                availableTiles = len(list(self.game.availableTiles(moved)))
                
                depth = 28//availableTiles+1
                moves.append((move, self.numberOfLossesFrom(board, depth)))
            else:
                pass # not gonna use useless moves
        
        moves.sort(key=lambda x: x[1])
        return moves[0][0]


    def numberOfLossesFrom(self, board, depth):
        key = str(board)
        if key in self.cache:
            return self.cache[key]
        
        lost = self.game.verifyLoss(board)
        if depth == 0 or lost:
            self.cache[str(board)] = lost
            return lost
        # else...

        score = 0

        for move in range(3):
            moved, _ = self.game.transform(board, move)

            if np.array_equal(board, moved):
                continue # nothing to do, not gonna count useless moves
            else:
                tiles = list(self.game.availableTiles(moved))
                if len(tiles) <= 3:
                    samples = len(tiles)
                elif 3 < len(tiles) <= 8:
                    samples = len(tiles) // 2
                else:
                    samples = 4
                for coord in choices(tiles, k=samples):
                    x, y = coord
                    
                    moved[x,y] = 4
                    score += self.numberOfLossesFrom(moved, depth-1)

                    moved[x,y] = 2
                    score += 9*self.numberOfLossesFrom(moved, depth-1)
                    
                    moved[x,y] = 0
        
        self.cache[str(board)] = score

        return score

In [ ]:
env = GameOf2048()
agent = DTAgent()

controller = Control(sleepTime=0.1, verbose=True) 

for _ in range(500):
    controller.pressArrow(agent.nextMove(controller.currentGrid))
    controller.updateGrid()
    if env.verifyLoss(controller.currentGrid):
        break
